# Variational Autoencoder for Collaborative Filtering*

## 1.Model

<br>u $\epsilon$ {1,...,U}
<br>i $\epsilon$ {1,...,I}
<br> User-item interaction matrix: $X \, \epsilon \, \mathbb{N}^{U \times I}$ where $x_u=[x_{u1},...,x_{uI}]^T \, \epsilon \, \mathbb{N}^I$

<br>$z_u$: K-dimensional hidden factor for user u
<br>$f_{\theta}(.)$: non-linear function $\epsilon \, \mathbb{R}^I$
<br>$\pi(z_u)$: the probability function from which $x_u$ vector is assumed to be drawn
<br>$N_u=\sum_{i} x_{ui}$: Total number of clicks given the user u

$$z_u \sim N(0,I_k)$$
$$\pi(z_u) \propto exp(\, f_{\theta}(z_u)\,)$$
$$x_u \sim Mult(N_u, \pi(z_u))$$

Log-likelihood function: $log\, p_{\theta}(x_u | z_u)=\sum_i x_{ui} \, log \, \pi_i(z_u)$
For comparison:
1. Gaussian Log-likelihood
2. Logistic log-likelihood

## 2. Variational Inference

To learn the generative model we need to estimate the parameter $\theta$ of the non-linear function $f_{\theta}(.)$. To do so, the intractable posterior $p(z_u |x_u)$ needs to be calculated for each data point. Variational inference approximates the true posterior to an instrumental posterior $q(z_u)$ with Kullback-Leibler divergence $KL(q(z_u) || p(z_u |x_u))$ where,
$$q(z_u) \sim \mathcal{N}(\mu_u,diag\{\sigma^2_u\})$$

The objective of variational inference to select $(\mu_u, \sigma^2_u)$ so that $KL(q(z_u) || p(z_u |x_u))$ is minimized.

### 2.1.Amortized inference and variational autoencoders

The number of variational parameters $(\mu_u, \sigma^2_u)$ grows with the number of users and items in the dataset. This is a bottleneck for commercial recommendation systems, since there are millions of users and items. To cope with this issue, variational autoencoders are used which are in fact data-dependent function. This is commonly called the inference model:
$$g_{\phi}(x_u)=[\mu_{\phi}(x_u), \sigma_{\phi}(x_u)] \: \epsilon \: \mathbb{R}^{2K}$$
Thus the variational distribution becomes also data-dependent:
$$q_{\phi}(z_u |x_u) \sim \mathcal{N}(\mu_{\phi}(x_u), diag\{\sigma^2_{\phi}(x_u)\})$$

Variational autoencoders ease the new inference problems analyzing user preferences by exploiting the similarity patterns inferred from past experiences.

*Learning VAEs:

\begin{align*}
KL(q_{\phi}(z_u |x_u) || p_{\theta}(z_u |x_u)) &= \mathbb{E}_q [log\,q_{\phi}(z_u |x_u)-log \, p_{\theta}(z_u |x_u) ] \qquad  \textbf{Eq.(1)} \\
&=\mathbb{E}_q [log\,q_{\phi}(z_u |x_u)-log \, p_{\theta}(z_u ,x_u)+ log\, p_{\theta}(x_u)] \\
&=\mathbb{E}_q [log\,q_{\phi}(z_u |x_u)-log \, p_{\theta}(x_u|z_u) -log\, p(z_u)] +log \, p_{\theta}(x_u)
\end{align*}

The evidence $p(x_u)$ is a constant term, and KL divergence is a non negative term, we can rearrange the equation as below:

$$log\, p_{\theta}(x_u)\geq \mathbb{E}_q[log\, p_{\theta}(x_u|z_u)]-KL(q_{\phi}(z_u|x_u)||p(z_u)) \equiv \mathcal{L}(x_u;\theta, \phi) \qquad \textbf{Eq.(2)}$$

$\mathcal{L}(x_u;\theta, \phi)$ is called evidence lower bound(ELBO).


Minimizing KL term in Eq.(1) is the same thing as maximizing ELBO.

*Reparameterization trick:

We can estimate the ELBO by sampling $z_u \sim q_{\phi}$ and make gradient ascent to optimize it. However, we can not take gradient of $q_{\phi}$ w.r.t. $\phi$. So we do the reparameterization trick:

1. Sample $\epsilon \sim \mathcal{N}(0,I_k)$
2. Reparameterize $z_u$ such that $z_u=[\mu_{\phi}(x_u),\epsilon \odot \sigma_{\phi}(x_u)]$

####  Algorithm I: VAE-SGD

<br>$\textbf{Input}$:Click matrix $X \, \epsilon \, \mathbb{N}^{I \times U}$
<br>Initialize $\phi$ and $\theta$
<br>$\textbf{While}$ $\textit{not converged}$ $\textbf{do}$
<br>$\:\:$sample a batch of users $\mathcal{U}$
<br>$\:\:$$\textbf{for}$ all $u \, \epsilon \, \mathcal{U}$ $\textbf{do}$
<br>$\:\:\:$sample $\epsilon \, \sim \, \mathcal{N}(0, I_k)$ and calculate $z_u$
<br>$\:\:\:$compute noisy gradient $\nabla_{\phi} \mathcal{L}$ and $\nabla_{\theta} \mathcal{L}$
<br>$\:\:$$\textbf{end}$
<br>$\:\:$Average the noisy gradients over batch
<br>$\:\:$Update $\phi$ and $\theta$ using gradient steps
<br>$\textbf{end}$

### 2.2.Alternative Interpretation of ELBO

We now define a new ELBO which interpretes first term in Eq.(2) as (negative) reconstruction error and the second term, KL term, as the regularization term.

$$\mathcal{L}_{\beta}(x_u;\theta, \phi)=\mathbb{E}_{q_{\phi}(z_u|x_u)}[log\, p_{\theta}(x_u|z_u)]-\beta \, \dot \,KL(q_{\phi}(z_u|x_u)||p(z_u)) $$

This regularization view of ELBO trades off between how well we can fit the data and how close the approximate posterior stays to the prior.

Selecting $\textbf{$\beta$}$: Using KL-annealing
